In [24]:
import json
import pymupdf
from PIL import Image

In [ ]:
# Define the text you are looking for
search_text = ["Lysvidde", "Fyrnr.", "Kartnr."]

# Open the PDF file
pdf_path = "Fyrliste_HeleLandet.pdf"
doc = pymupdf.open(pdf_path)

# Iterate over all pages
#for page_num in range(len(doc)):
for page_num in range(140):
    page = doc.load_page(page_num)
    
    # Extract text from the page
    text = page.get_text()

    # Check if the page contains the search text
    should_parse_page = all(map(lambda needle: needle in text, search_text))
    if should_parse_page:
        print(f"Text found on page {page_num + 1}")
        
        # Convert the page to an image
        pix = page.get_pixmap(dpi=200)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Save the image (optional)
        img_path = f"pages/page_{page_num + 1}.png"
        img.save(img_path, quality=100)

# Close the PDF document
doc.close()


In [21]:
def extract_lighthouses_using_gpt(image_file_name):
    import requests
    import os
    import json
    import base64
    file = open(image_file_name, "rb")
    image_bytes = file.read()
    base64_str = "data:image/jpeg;base64," + base64.b64encode(image_bytes).decode("utf-8")
    
    openai_api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + os.getenv('OPENAI_API_KEY', ''),
    }
    
    json_data = {
        'model': 'gpt-4o-2024-08-06',
        'messages': [
            {
                'role': 'system',
                'content': [
                    {
                        'type': 'text',
                        'text': 'Generate JSON for these lighthouses. When reading sectors, use the sector color column with a single letter. Ensure that all sectors are included.',
                    },
                ],
            },
            {
                'role': 'user',
                'content': [
                    {
                        'type': 'image_url',
                        'image_url': {
                            'url': base64_str
                        },
                    },
                ],
            },
        ],
        'response_format': {
            'type': 'json_schema',
            'json_schema': {
                'name': 'lighthouse_response',
                'strict': True,
                'schema': {
                    '$schema': 'http://json-schema.org/draft-07/schema#',
                    'type': 'object',
                    'properties': {
                        'items': {
                            'type': 'array',
                            'items': {
                                'type': 'object',
                                'properties': {
                                    'latitude': {
                                        'type': 'object',
                                        'properties': {
                                            'degrees': {
                                                'type': 'integer',
                                                'description': 'Degrees of latitude, ranging from -90 to 90.',
                                            },
                                            'minutes': {
                                                'type': 'number',
                                                'description': 'Minutes of latitude, ranging from 0 to 60.',
                                            },
                                        },
                                        'required': [
                                            'degrees',
                                            'minutes',
                                        ],
                                        'additionalProperties': False,
                                    },
                                    'longitude': {
                                        'type': 'object',
                                        'properties': {
                                            'degrees': {
                                                'type': 'integer',
                                                'description': 'Degrees of longitude, ranging from -180 to 180.',
                                            },
                                            'minutes': {
                                                'type': 'number',
                                                'description': 'Minutes of longitude, ranging from 0 to 60.',
                                            },
                                        },
                                        'required': [
                                            'degrees',
                                            'minutes',
                                        ],
                                        'additionalProperties': False,
                                    },
                                    'pattern': {
                                        'type': 'string',
                                    },
                                    'description': {
                                        'type': 'string',
                                    },
                                    'heightOverGround': {
                                        'type': 'number',
                                    },
                                    'height': {
                                        'type': 'number',
                                    },
                                    'sectors': {
                                        'type': 'array',
                                        'items': {
                                            'type': 'object',
                                            'properties': {
                                                'color': {
                                                    'type': 'string',
                                                },
                                                'start': {
                                                    'type': 'number',
                                                },
                                                'stop': {
                                                    'type': 'number',
                                                },
                                                'description': {
                                                    'type': 'string',
                                                },
                                            },
                                            'required': [
                                                'color',
                                                'start',
                                                'stop',
                                                'description',
                                            ],
                                            'additionalProperties': False,
                                        },
                                    },
                                    'area': {
                                        'type': 'string',
                                    },
                                    'name': {
                                        'type': 'string',
                                    },
                                    'location': {
                                        'type': 'string',
                                    },
                                    'maxRange': {
                                        'type': 'number',
                                    },
                                },
                                'required': [
                                    'latitude',
                                    'longitude',
                                    'pattern',
                                    'description',
                                    'heightOverGround',
                                    'height',
                                    'sectors',
                                    'name',
                                    'location',
                                    'area',
                                    'maxRange',
                                ],
                                'additionalProperties': False,
                            },
                        },
                    },
                    'required': [
                        'items',
                    ],
                    'additionalProperties': False,
                },
            },
        },
        'temperature': 0.0,
        'max_tokens': 4096,
    }
    
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=json_data)
    data = response.json()
    lighthouses_on_page_str = data['choices'][0]['message']['content']
    lighthouses_on_page = json.loads(lighthouses_on_page_str)
    return lighthouses_on_page['items']


In [50]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models
import requests
import os
import json
import base64
    
def extract_lighthouses_using_gemini(image_file_name):
    file = open(image_file_name, "rb")
    image_bytes = file.read()

    vertexai.init(project="cognitedata-development", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
    )
    chat = model.start_chat()

    image = Part.from_data(mime_type="image/png", data=image_bytes)

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json",
        "response_schema": {
            'type_': 'OBJECT',
            'properties': {
                'items': {
                    'type_': 'ARRAY',
                    'items': {
                        'type_': 'OBJECT',
                        'properties': {
                            'latitude': {
                                'type_': 'OBJECT',
                                'properties': {
                                    'degrees': {
                                        'type_': 'INTEGER',
                                        'description': 'Degrees of latitude, ranging from -90 to 90.',
                                    },
                                    'minutes': {
                                        'type_': 'NUMBER',
                                        'description': 'Minutes of latitude, ranging from 0 to 60.',
                                    },
                                },
                                'required': [
                                    'degrees',
                                    'minutes',
                                ]
                            },
                            'longitude': {
                                'type_': 'OBJECT',
                                'properties': {
                                    'degrees': {
                                        'type_': 'INTEGER',
                                        'description': 'Degrees of longitude, ranging from -180 to 180.',
                                    },
                                    'minutes': {
                                        'type_': 'NUMBER',
                                        'description': 'Minutes of longitude, ranging from 0 to 60.',
                                    },
                                },
                                'required': [
                                    'degrees',
                                    'minutes',
                                ]
                            },
                            'pattern': {
                                'type_': 'STRING',
                            },
                            'description': {
                                'type_': 'STRING',
                            },
                            'heightOverGround': {
                                'type_': 'NUMBER',
                            },
                            'height': {
                                'type_': 'NUMBER',
                            },
                            'sectors': {
                                'type_': 'ARRAY',
                                'items': {
                                    'type_': 'OBJECT',
                                    'properties': {
                                        'color': {
                                            'type_': 'STRING',
                                        },
                                        'start': {
                                            'type_': 'NUMBER',
                                        },
                                        'stop': {
                                            'type_': 'NUMBER',
                                        },
                                        'description': {
                                            'type_': 'STRING',
                                        },
                                    },
                                    'required': [
                                        'color',
                                        'start',
                                        'stop',
                                        'description',
                                    ]
                                },
                            },
                            'area': {
                                'type_': 'STRING',
                            },
                            'name': {
                                'type_': 'STRING',
                            },
                            'location': {
                                'type_': 'STRING',
                            },
                            'maxRange': {
                                'type_': 'NUMBER',
                            },
                        },
                        'required': [
                            'latitude',
                            'longitude',
                            'height',
                            'sectors',
                            'name',
                            'area',
                            'maxRange',
                        ]
                    },
                },
            },
            'required': [
                'items',
            ]
        },
    }

    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    }
    
    response = chat.send_message(
      [image, "Generate JSON for these lighthouses. 'Karakter' is used for flash pattern (e.g. Fl(3) W 30s). Max range is used as the max value of Lysvidde. When reading sectors, use the sector color column with a single letter. Ensure that all sectors are included."],
      generation_config=generation_config,
      safety_settings=safety_settings
    )
    return json.loads(response.candidates[0].content.text)['items']

In [51]:
def convert_pdf_page_to_image(page, image_file_name):
    pix = page.get_pixmap(dpi=200)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    img.save(image_file_name, quality=100)

def validate_text_existence(page_text, page_number, lighthouse):
    # TODO: in case the same coordinates appear more than once, we can verify
    # that the value appears the right amount of times (or more).
    errors = []
    
    needles = {
        "latitude_minutes": str(lighthouse['latitude']['minutes']),
        "latitude_degrees": str(lighthouse['latitude']['degrees']),
        "longitude_minutes": str(lighthouse['longitude']['minutes']),
        "longitude_degrees": str(lighthouse['longitude']['degrees'])
    }
    
    for sector_index, sector in enumerate(lighthouse['sectors']):
        start_str = str(sector['start']).replace(".", ",")
        stop_str = str(sector['stop']).replace(".", ",")
        needles[f"{sector_index}_start"] = start_str
        needles[f"{sector_index}_stop"] = stop_str

    minimum_value_count = {}
    for value in needles.values():
        if not value in minimum_value_count:
            minimum_value_count[value] = 0
        minimum_value_count[value] += 1
    
    for key, value in needles.items():
        value_count_on_page = page_text.count(value)
        if value_count_on_page < minimum_value_count[value]:
            errors.append(f"Value {key} missing at least once for {lighthouse['name']} on page {page_number+1} (value is {value} should appear {minimum_value_count[value]} times)")
    return errors

def validate_extracted_lighthouses(page, page_number, lighthouses):
    page_text = page.get_text()
    
    errors = []
    
    for lighthouse in lighthouses:
        errors.extend(validate_text_existence(page_text, page_number, lighthouse))
        
    return errors

def parse_lighthouses_for_page(document, page_number):
    image_file_name = "pages/page.png"
    page = document.load_page(page_number)
    
    convert_pdf_page_to_image(page, image_file_name)
    # lighthouses_on_page = extract_lighthouses_using_gpt(image_file_name)
    lighthouses_on_page = extract_lighthouses_using_gemini(image_file_name)
    
    # Since lower case L and upper case I looks similar,
    # the models confuse them sometimes. We don't. Since FI 
    # is not a valid flash pattern, replace with its valid Fl value.
    for lighthouse in lighthouses_on_page:
        if "pattern" in lighthouse and "FI" in lighthouse["pattern"]:
            lighthouse["pattern"].replace("FI", "Fl")
            
    errors = validate_extracted_lighthouses(page, page_number, lighthouses_on_page)
    return lighthouses_on_page, errors

# pdf_path = "Fyrliste_HeleLandet.pdf"
# document = pymupdf.open(pdf_path)
# lighthouses_on_page, errors = parse_lighthouses_for_page(document, 29)
# errors

In [52]:
all_errors = []
pdf_path = "Fyrliste_HeleLandet.pdf"
document = pymupdf.open(pdf_path)
for page_number in range(45, 65):
    print("Parsing page ", page_number)
    lighthouses_on_page, errors = parse_lighthouses_for_page(document, page_number)
    print(f"Found {len(errors)} errors on page {page_number+1}")
    all_errors.extend(errors)


Parsing page  45


KeyboardInterrupt: 

In [ ]:
# Also remember to check sector colors with their starting sector angle as it seemed like it could make mistakes sometimes. 
# This can be used to generate a list of potential problems we can/should manually fix before updating the lighthouses.json file.